# Обучение Word2Vec на своих данных

### Загружаем и готовим данные отзывов на банки

In [ ]:
!pip install pymorphy2[fast]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.1/371.1 KB 24.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for DAWG: filename=DAWG-0.8.0-cp39-cp39-linux_x86_64.whl size=1272146 sha256=7e5a33f1c8e689fefca64431646ba632beceb81854b50dcedf53ac08156e0961
  Stored in directory: /root/.cache/pip/wheels/ca/92/6b/42b536c832a800acdabe5ad23487a60206c1f46ae5a0a2c76d
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=6983358b9aec50bcdb7e0d25734dc3266b121d07e3a7172223e788896abd04de
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built DAWG docopt


In [ ]:
import pandas as pd
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def preprocess(text, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return preprocessed_text

In [ ]:
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...']
stop_words = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

In [ ]:
!wget https://www.dropbox.com/s/a9r0b2yj3vqvi13/banks.csv?dl=1 -O banks.csv

--2023-04-08 11:14:43--  https://www.dropbox.com/s/a9r0b2yj3vqvi13/banks.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/a9r0b2yj3vqvi13/banks.csv [following]
--2023-04-08 11:14:43--  https://www.dropbox.com/s/dl/a9r0b2yj3vqvi13/banks.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca12a7af19d1bf5b2f95791cc46.dl.dropboxusercontent.com/cd/0/get/B5wQKvKWFNEK3TYUHVQkqPzdXGeHLBV_JbXzFZ52nqv6Pqb-MZNSMEViplDKeDuObdBkRwdDT7wrnhMvbMD2QuVNJV92kuiVfcH9UFSQkmoJDwnfA4JmEU2aO0fW3XcImpwiumqql7e2jNHk5kc0_ZJGqzhRkoM4xqGfsv-8pikNUA/file?dl=1# [following]
--2023-04-08 11:14:43--  https://uca12a7af19d1bf5b2f95791cc46.dl.dropboxusercontent.com/cd/0/get/B5wQKvKWFNEK3TYUHVQkqPzdXGeHLBV_JbXzFZ52nqv6Pqb-MZNSMEViplDKeDuObdBkRwdDT7wrnhMvbMD2QuV

In [ ]:
banks = pd.read_csv('banks.csv', sep='\t', index_col='idx');

In [ ]:
banks

,Score,Text
idx,,
0,Positive,В Альфа-Банке работает замечательная девушка -...
1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...
2,Positive,Очень порадовала оперативность работы в банке....
3,Negative,Имела неосторожность оформить потреб. кредит в...
4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...
...,...,...
13994,Positive,"О высокой надёжности МКБ, порядочности и добро..."
13995,Positive,"Обслуживаюсь в офисе на Чернореченской 42а, ка..."
13996,Positive,Попала сегодня в очень неприятную ситуацию. Ре...


In [ ]:
banks['Preprocessed_texts'] = banks.apply(lambda row: preprocess(row['Text'], punctuation_marks, stop_words, morph), axis=1)

In [ ]:
banks

,Score,Text,Preprocessed_texts
idx,,,
0,Positive,В Альфа-Банке работает замечательная девушка -...,"[альфа-банк, работать, замечательный, девушка,..."
1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...,"[оформлять, рассрочка, м., видео, мег, тёплый,..."
2,Positive,Очень порадовала оперативность работы в банке....,"[очень, порадовать, оперативность, работа, бан..."
3,Negative,Имела неосторожность оформить потреб. кредит в...,"[иметь, неосторожность, оформить, потреба, кре..."
4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...,"[небольшой, предыстория, найти, сайт, mdm, бан..."
...,...,...,...
13994,Positive,"О высокой надёжности МКБ, порядочности и добро...","[высокий, надёжность, мкб, порядочность, добро..."
13995,Positive,"Обслуживаюсь в офисе на Чернореченской 42а, ка...","[обслуживаться, офис, чернореченский, 42а, физ..."
13996,Positive,Попала сегодня в очень неприятную ситуацию. Ре...,"[попасть, сегодня, очень, неприятный, ситуация..."


## Обучение Word2Vec на отзывах на банки

In [ ]:
import gensim.models

In [ ]:
model = gensim.models.Word2Vec(sentences=banks['Preprocessed_texts'],
                               # Используем только слова, встречающиеся минимум 5 раз и в любом случае, если слово используется редко, то сложно понять его смысл
                               min_count=5,
                               vector_size=100)

In [ ]:
model.wv['альфа-банк']

array([-0.25089076, -0.65877676,  0.5147335 ,  0.92979896, -1.064438  ,
       -1.1806476 ,  0.22694483,  0.89967453, -0.5941724 ,  1.0317999 ,
        1.0201598 , -0.22397493, -0.5596784 , -0.88198763, -0.43373162,
       -0.5547372 , -0.09502857, -0.43976268, -0.1576873 , -0.485755  ,
       -1.1975471 ,  0.7229492 ,  0.52628785,  0.05852138,  0.13857621,
       -1.6400115 , -1.0203139 ,  0.72240645, -0.7048731 , -0.5188238 ,
       -0.17077187, -0.7935835 ,  1.3439844 , -0.55809987, -1.5541853 ,
       -0.9900477 ,  1.5649211 , -0.9317926 ,  0.35660362, -1.1632097 ,
        0.7336595 ,  0.9510679 , -0.05715861,  0.7839336 ,  0.37505236,
        1.3531667 ,  0.74608594,  0.5400542 ,  0.02779879,  0.42284262,
       -0.6549035 ,  0.22599271,  0.6500962 , -1.1732337 ,  0.20349202,
       -0.77509016,  1.5029048 ,  0.44757253, -0.43032676, -0.02084845,
       -0.95356554, -0.3573223 , -0.17432714,  0.38665158,  1.033842  ,
        0.47573698,  0.7818817 ,  1.7582484 , -0.41403112, -0.49

In [ ]:
model.wv['работать']

array([-0.10175087, -1.0809928 ,  2.108006  , -2.486412  ,  0.9196213 ,
       -1.4626027 , -0.21060044,  0.8756251 , -1.1829846 ,  0.29253986,
        0.9716915 , -1.8375353 ,  0.21940854,  1.0647212 , -1.3188096 ,
       -2.0270655 ,  1.5214329 , -1.4050672 , -0.998928  ,  0.4807411 ,
        1.5911163 ,  0.06308132, -0.5794307 , -0.29872635,  0.4041926 ,
        1.3993962 ,  1.1623596 , -0.92349285, -0.15602101, -1.2537895 ,
       -0.91006106, -0.6138517 , -0.07856853,  1.4482253 ,  0.66916853,
        2.3700216 ,  1.4052576 , -0.20275462,  2.3438592 , -1.0315827 ,
       -0.59384686,  0.50704604,  1.8725182 , -0.74886465, -2.1523476 ,
        0.74585223, -0.75162435, -0.37324363, -1.0254221 , -0.91119355,
        0.10711198, -0.55019647, -0.8051673 ,  0.6939517 ,  0.10643438,
       -0.3512112 ,  0.04656842,  0.21654834, -2.7612953 , -0.19400752,
        0.741294  , -1.7558118 , -0.09971482, -1.0906675 ,  0.06469075,
       -2.9385962 ,  0.49728748, -0.858944  ,  0.47176296,  1.00

In [ ]:
model.wv['замечательный']

array([-0.8730825 , -0.47168142,  1.0432241 , -0.6416336 ,  0.05238728,
       -0.5432315 ,  1.1219734 , -0.2129176 ,  0.02228874,  0.02450718,
       -0.9726496 , -0.8511167 ,  0.40730917, -0.548813  ,  0.2380059 ,
       -0.4824895 , -0.06937856,  0.04602629, -0.23005596, -0.80491364,
        0.6730905 , -0.13042723,  0.544956  , -0.28090075, -0.2804369 ,
        0.53933316,  0.06290711,  0.25830248, -0.3058899 ,  0.18012337,
       -0.38275272, -0.41012093, -0.753202  ,  0.26304454, -0.74885386,
        0.52303267,  0.22779746, -0.6058196 ,  0.01166278, -0.73342   ,
       -0.05113549,  0.4113305 ,  0.2075743 , -0.65207016, -0.0584805 ,
       -0.05863456,  0.02648681,  0.57240814, -0.20739637,  0.60122174,
        0.13498734, -1.1715037 , -0.47627613,  0.08168343, -0.47841012,
        0.14343655,  0.5263094 ,  0.6050259 , -0.9302115 ,  0.85871226,
       -0.7627511 , -1.0218766 , -0.12033543, -0.7765567 ,  0.20139004,
        0.6807579 , -0.17919402, -0.0060102 ,  0.3547247 ,  0.47

In [ ]:
model.wv.most_similar('замечательный')

[('прекрасный', 0.757600724697113),
 ('отличный', 0.7105039358139038),
 ('достойный', 0.6932812929153442),
 ('молодец', 0.69072026014328),
 ('псб', 0.6794296503067017),
 ('тёплый', 0.6666871905326843),
 ('великолепный', 0.6629226207733154),
 ('клиентоориентировать', 0.6602783203125),
 ('позитивный', 0.6602382063865662),
 ('выручить', 0.6591485738754272)]

In [ ]:
model.wv.most_similar('стрессовый')

[('конфликтный', 0.8973307013511658),
 ('прояснение', 0.8952047228813171),
 ('абсурдность', 0.8785838484764099),
 ('нештатный', 0.8760561943054199),
 ('безвыходный', 0.8679152131080627),
 ('абсурдный', 0.8563932180404663),
 ('затруднительный', 0.8544430136680603),
 ('тупиковый', 0.8543331623077393),
 ('разбор', 0.8514256477355957),
 ('критический', 0.8501579165458679)]

## Сохранение обученной модели

In [ ]:
model.save('word2vec-banki.ru-50')

In [ ]:
!ls -lh

total 51M
-rw-r--r-- 1 root root  39M Apr  8 11:14 banks.csv
drwxr-xr-x 1 root root 4.0K Apr  6 13:39 sample_data
-rw-r--r-- 1 root root  12M Apr  8 11:27 word2vec-banki.ru-50


## Загрузка сохраненной модели

In [ ]:
new_model = gensim.models.Word2Vec.load('word2vec-banki.ru-50')

In [ ]:
new_model.wv['работать']

array([-0.10175087, -1.0809928 ,  2.108006  , -2.486412  ,  0.9196213 ,
       -1.4626027 , -0.21060044,  0.8756251 , -1.1829846 ,  0.29253986,
        0.9716915 , -1.8375353 ,  0.21940854,  1.0647212 , -1.3188096 ,
       -2.0270655 ,  1.5214329 , -1.4050672 , -0.998928  ,  0.4807411 ,
        1.5911163 ,  0.06308132, -0.5794307 , -0.29872635,  0.4041926 ,
        1.3993962 ,  1.1623596 , -0.92349285, -0.15602101, -1.2537895 ,
       -0.91006106, -0.6138517 , -0.07856853,  1.4482253 ,  0.66916853,
        2.3700216 ,  1.4052576 , -0.20275462,  2.3438592 , -1.0315827 ,
       -0.59384686,  0.50704604,  1.8725182 , -0.74886465, -2.1523476 ,
        0.74585223, -0.75162435, -0.37324363, -1.0254221 , -0.91119355,
        0.10711198, -0.55019647, -0.8051673 ,  0.6939517 ,  0.10643438,
       -0.3512112 ,  0.04656842,  0.21654834, -2.7612953 , -0.19400752,
        0.741294  , -1.7558118 , -0.09971482, -1.0906675 ,  0.06469075,
       -2.9385962 ,  0.49728748, -0.858944  ,  0.47176296,  1.00

## Полезные ссылки

1. [Библиотека Gensim](https://radimrehurek.com/gensim/index.html).
2. [Word2Vec в Gensim](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html).
